In [26]:
import json

with open('Mock_data.json') as json_data:
    mock_data = json.load(json_data)
    json_data.close()

def write_to_json(dict=mock_data,filepath='Mock_data.json'):
    with open(filepath,'w') as file:
        json.dump(dict, file,indent=4)
        file.close()

In [27]:


e = 2.7182818

def sigmoid(x):
    return (e**x) / (1 + (e**x))

def tanh(x):
    return ((e**x) - (e**-x))/ ((e**x)+(e**-x))

def normalize(array,natural_numbers=True):
    min = array[0]
    max = array[0]
    for n in array[1:]:
        if n < min: min = n
        if n > max: max = n

    result = {
        "min":min,
        "max":max,
        "data":[]
    }
    for n in array:
        n = (n-min)/(max-min)
        if not natural_numbers:
            n = (2*n)-1
        result['data'].append(n)

    return result

def denormalize(array,max,min,natural_numbers=True):
    result = []
    for n in array:
        if not natural_numbers:
            n = (n + 1)/2
        n = (n*(max-min))+min
        result.append(n)
    return result


In [ ]:
class LSTM:

    def __init__(self):

        self.cell_state = 0
        self.short = 0

        # variables for forgetting long-term memory
        self.FG_input_weight = 1
        self.FG_short_weight = 1
        self.FG_bias = 0

        # variables for input gate
        self.IG_input_weight = 1
        self.IG_short_weight = 1
        self.IG_bias = 0
        # forget gate within input gate
        self.IGF_input_weight = 1
        self.IGF_short_weight = 1
        self.IGF_bias = 0

        # variables for forget gate within output gate
        self.OGF_input_weight = 1
        self.OGF_short_weight = 1
        self.OGF_bias = 0


    ## returns forget gate modifier
    def forget_gate(self,input,input_weight=1,short_weight=1,bias=0):
        weighted_input = input*input_weight
        weighted_short = self.short*short_weight
        sum = weighted_short + weighted_input + bias
        result = sigmoid(sum)
        return result

    ## returns value to add to cell state
    def input_gate(self,input):
        weighted_input = input * self.IG_input_weight
        weighted_short = self.short * self.IG_short_weight
        sum = weighted_short + weighted_input + self.IG_bias
        forget_modifier = self.forget_gate(input, self.IGF_input_weight, self.IGF_short_weight, self.IGF_bias)
        result = tanh(sum) * forget_modifier
        return result

    ## returns new hidden state
    def output_gate(self,input):
        forget_modifier = self.forget_gate(input, self.OGF_input_weight, self.OGF_short_weight, self.OGF_bias)
        output = tanh(self.cell_state) * forget_modifier
        return output
    
    def forward(self,input):
        self.cell_state *= self.forget_gate(input, self.FG_input_weight, self.FG_short_weight, self.FG_bias)
        self.cell_state += self.input_gate(input)
        self.short = self.output_gate(input)
        print("Pass Complete:",
            "\n\tInput -> ",input,
            "\n\tNew Short -> ",self.short,
            "\n\tNew Long -> ",self.cell_state)

    # inputs should be normalized 
    def training_step(self,epoch):
        for time_step in epoch:
            self.forward(time_step)
        print("*Sequence Unroll Complete*")



In [ ]:
def main():
    model = LSTM()
    model.training_step(mock_data["normalized_-1to1"]["data"])
    print("Prediction: ",[model.short],"---->",denormalize([model.short],23,5,False))
main()

Input Unrolled: 
	Input ->  -0.5555555555555556 
	New Short ->  -0.06633219685645585 
	New Long ->  -0.18399166560155847
Input Unrolled: 
	Input ->  -1.0 
	New Short ->  -0.06247054020603262 
	New Long ->  -0.24894755799539375
Input Unrolled: 
	Input ->  0.11111111111111116 
	New Short ->  -0.052368031275773816 
	New Long ->  -0.10260840283560163
Input Unrolled: 
	Input ->  0.7777777777777777 
	New Short ->  0.22592243298435377 
	New Long ->  0.3487847939286669
Input Unrolled: 
	Input ->  -0.7777777777777778 
	New Short ->  -0.020427266010980043 
	New Long ->  -0.05595696822618346
Input Unrolled: 
	Input ->  0.0 
	New Short ->  -0.018698382658411384 
	New Long ->  -0.037800644431316976
Input Unrolled: 
	Input ->  -0.33333333333333337 
	New Short ->  -0.06358581553944426 
	New Long ->  -0.1552369873116827
Input Unrolled: 
	Input ->  -0.6666666666666667 
	New Short ->  -0.08058154610605835 
	New Long ->  -0.2531069108281762
Input Unrolled: 
	Input ->  0.11111111111111116 
	New Short ->  